## Image classification with Convolutional Neural Networks

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline
BASE_DATA_DIR = "/Users/HitMan/External/ai_data/data/"
BASE_DATA_DIR = "/Users/HitMan/Desktop/"
PATH = BASE_DATA_DIR + "dogscats/"
%ls $PATH

models/ sample/ test1/  tmp/    train/  valid/


In [2]:
# This file contains all the main external libs we'll use
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [3]:
# PATH = "data/dogscats/"
sz=224
arch=vgg16
bs=64

In [4]:
??vgg16

Signature: vgg16(pre)
Source:    def vgg16(pre): return children(vgg16_bn(pre))[0]
File:      ~/workspace/home/fastai/courses/dl1/fastai/torch_imports.py
Type:      function


In [5]:
??ImageClassifierData

Init signature: ImageClassifierData(path, datasets, bs, num_workers, classes)
Source:        
class ImageClassifierData(ImageData):
    @property
    def is_multi(self): return self.trn_dl.dataset.is_multi

    @staticmethod
    def get_ds(fn, trn, val, tfms, test=None, **kwargs):
        res = [
            fn(trn[0], trn[1], tfms[0], **kwargs), # train
            fn(val[0], val[1], tfms[1], **kwargs), # val
            fn(trn[0], trn[1], tfms[1], **kwargs), # fix
            fn(val[0], val[1], tfms[0], **kwargs)  # aug
        ]
        if test is not None:
            test_lbls = np.zeros((len(test),1))
            res += [
                fn(test, test_lbls, tfms[1], **kwargs), # test
                fn(test, test_lbls, tfms[0], **kwargs)  # test_aug
            ]
        else: res += [None,None]
        return res

    @classmethod
    def from_arrays(cls, path, trn, val, bs=64, tfms=(None,None), classes=None, num_workers=4, test=None):
        """ Read in images and their labels

In [6]:
# Uncomment the below if you need to reset your precomputed activations
# !rm -rf {PATH}tmp

In [7]:
??ConvLearner

Init signature: ConvLearner(data, models, precompute=False, **kwargs)
Source:        
class ConvLearner(Learner):
    def __init__(self, data, models, precompute=False, **kwargs):
        self.precompute = False
        super().__init__(data, models, **kwargs)
        self.crit = F.binary_cross_entropy if data.is_multi else F.nll_loss
        if data.is_reg: self.crit = F.l1_loss
        elif self.metrics is None:
            self.metrics = [accuracy_thresh(0.5)] if self.data.is_multi else [accuracy]
        if precompute: self.save_fc1()
        self.freeze()
        self.precompute = precompute

    @classmethod
    def pretrained(cls, f, data, ps=None, xtra_fc=None, xtra_cut=0, **kwargs):
        models = ConvnetBuilder(f, data.c, data.is_multi, data.is_reg, ps=ps, xtra_fc=xtra_fc, xtra_cut=xtra_cut)
        return cls(data, models, **kwargs)

    @property
    def model(self): return self.models.fc_model if self.precompute else self.models.model

    @property
    def data(self): r

In [8]:
??transforms_side_on

Type:        list
String form: [<fastai.transforms.RandomRotate object at 0x11aa81320>, <fastai.transforms.RandomLighting object at 0x11aa812b0>, <fastai.transforms.RandomFlip object at 0x11aa81390>]
Length:      3


In [17]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))

In [ ]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /Users/HitMan/.torch/models/vgg16_bn-6c64b313.pth
100%|██████████| 553507836/553507836 [01:23<00:00, 7463886.16it/s]


  0%|          | 0/360 [00:00<?, ?it/s]

In [10]:
learn.fit(0.01, 3, cycle_len=1)

A Jupyter Widget

[ 0.       0.05266  0.02898  0.99072]                         
[ 1.       0.04534  0.02512  0.99023]                         
[ 2.       0.04645  0.02377  0.99121]                         



In [4]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)

In [5]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=4)
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [6]:
learn.fit(1e-2, 2)

A Jupyter Widget

[ 0.       0.05622  0.0268   0.98926]                         
[ 1.       0.04222  0.02508  0.99072]                          



In [7]:
learn.precompute=False

In [8]:
learn.fit(1e-2, 1, cycle_len=1)

A Jupyter Widget

[ 0.       0.05511  0.02654  0.98909]                         



In [8]:
learn.unfreeze()

In [9]:
lr=np.array([1e-4,1e-3,1e-2])

In [10]:
learn.fit(lr, 1, cycle_len=1)

A Jupyter Widget

[ 0.       0.05664  0.0245   0.99023]                         



In [11]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

A Jupyter Widget

[ 0.       0.05955  0.02217  0.9917 ]                         
[ 1.       0.03902  0.0181   0.99414]                         
[ 2.       0.03676  0.0168   0.99414]                         
[ 3.       0.03097  0.01607  0.99316]                         
[ 4.       0.02823  0.01574  0.99316]                         
[ 5.       0.02663  0.01448  0.99316]                         
[ 6.       0.03113  0.01642  0.99414]                         



In [14]:
learn.fit(lr, 3, cycle_len=3)

A Jupyter Widget

[ 0.       0.02769  0.01896  0.99268]                         
[ 1.       0.01939  0.01417  0.99365]                         
[ 2.       0.01935  0.0142   0.99414]                         
[ 3.       0.02465  0.01641  0.99316]                         
[ 4.       0.02753  0.01376  0.99365]                         
[ 5.       0.01721  0.01413  0.99414]                         
[ 6.       0.0221   0.01317  0.99365]                         
[ 7.       0.01789  0.01379  0.99268]                         
[ 8.       0.01861  0.0139   0.99316]                         



In [15]:
log_preds,y = learn.TTA()
accuracy(log_preds,y)

0.995